# Центроидная декомпозиция

* Обобщение метода «разделяй-и-властвуй» на деревья
* Очень часто помогает отвечать на запросы «сколько есть путей с такими-то свойствами»
* Также можно отвечать на запросы «есть ли такое-то свойства у этого пути»
* Вполне может встретиться как последняя задача Открытой или Всероса, или как div1 C и дальше
* Иногда вместо неё можно написать HLD (сложнее) или сливающиеся сеты (легче)

![](https://qph.ec.quoracdn.net/main-qimg-4728cdabd3052afdcb34cde75df8e97d.webp)

**Определение**. Центром (centroid) дерева будем называть вершину, при удалении которой размеры оставшихся компонент будут не более $\frac{n}{2}$.

Центр всегда существует. Иногда центров два. Искать можно так:

In [ ]:
int s[maxn];

int size (int v) {
    s[v] = 1;
    for (int u : g[v])
        s[v] += sizes(u);
    return s[v];
}

// второй параметр — размер дерева
int centroid (int v, int n) {
    for (int u : g[v])
        if (s[u] > n/2)
            return centroid(u, n);
    return v;
}

* `centroid` вернет вершину, у которой размеры всех детей не больше $\frac{n}{2}$ (явно проверяется в `if`-е)
* Мы пришли в эту вершину, когда её размер был больше $\frac{n}{2}$ $\implies$ в «обратном» направлении не больше $n - (\frac{n}{2}+1) = \frac{n}{2}-1$ вершин

Значит, размеры всех соседей не больше половины $n$, и алгоритм корректен.

**Определение**. Центроидной декомпозицией будем называть рекурсивный процесс «выделить центроид, удадить, запуститься от компонент».

Заметим, что каждая вершина ровно один раз будет центроидом — в конце всё дерево будет удалено.

**Определение**. Компонентой центроида будем называть множество вершин, достижимых из центроида непосредственно перед его удалением.

## Зачем мы всё это делали:

**Утверждение**. Каждая вершина входит в $O(\log n)$ компонент.

*Доказательство*. Центроид разбивает дерево на компоненты в два раза меньшего размера.

**Утверждение**. Для любого пути $a \leadsto b$ есть единственный центроид $c$, в чьей компоненте были и $a$, и $b$.

*Доказательство*. Каждая вершина когда-то была центроидом. Какая-то из вершин на пути была центроидом первой, и она навсегда разъединила $a$ и $b$.

Эти факты можно использовать для подсчета количества путей с какими-то свойствами или ответа на запросы про пути. Рассмотрим пример: задача «число путей заданной длины».

## Подсчет путей с заданным свойством

Для каждого центроида перед его удалением будем прибавлять к ответу число интересующих нас путей, которые проходят через этот центроид. Количество таких путей можно посчитать за размер текущей компоненты. Асимптотика $O(n \log n)$, потому что каждая вершина входит в $O(\log n)$ компонент.

Чтобы не делать явно удаление вершины, заведем массив `used[]` — была ли вершина удалена.

In [ ]:
int l = 179; // нужная нам длина
int ans = 0; // тут мы храним ответ

bool used[maxn];
int s[maxn];

void sizes (int v, int p) {
    s[v] = 1;
    for (int u : g[v])
        if (u != p && !used[u])
            sizes(u, v), s[v] += s[u];
}

int centroid (int v, int p, int n) {
    for (int u : g[v])
        if (u != p && !used[u] && s[u] > n/2)
            return centroid(u, v, n);
    return v;
}

// записывает в t[] глубины вершин
void dfs (int v, int p, int d, vector<int> &t) {
    t.push_back(d);
    for (int u : g[v])
        if (u != p && !used[u])
            dfs(u, v, t);
} 

void solve (int v) {
    /* <единственный зависящий от конкретной задачи код> */
    size(v);
    vector<int> d(s[v], 0);
    d[0] = 1;
    for (int u : g[v]) {
        if (!used[u]) {
            vector<int> t;
            dfs(u, v, 1, t);
            for (int x : t)
                if (x <= l)
                    ans += d[l-x];
            for (int x : t)
                d[x]++;
        }
    }
    /* </единственный зависящий от конкретной задачи код> */

    used[v] = 1;
    for (int u : g[v])
        if (!used[u])
            solve(centroid(u, v, s[u]/2));
}

Во многих задачах вместо центроидной декомпозиции можно использовать сливающиеся сеты, heavy-light декомпозицию или разные (часто персистентные) структуры данных. Конкретно здесь самое просто решение будет со сливающимися сетами с сохранением асимптотики $O(n \log n)$.

## Как структура данных

Мы находим что нам надо сразу, как получаем новую компоненту. Если мы у нас есть запросы посчитать что-то на пути, то мы часто можем обрабатывать их в offline, храня вместе с вершиной список ещё не отвеченных запросов, относящихся к ней. Таким образом, каждый запрос будет просмотрен $O(\log n)$ раз, пока не будет удален.

Альтернативно, можно сначала «построить» центроидную декомпозицию, а потом отвечать на запросы. Создадим двумерный массив `centroid[][]` размера $n \times \log n$, в котором для каждой вершины будем хранить $O(\log n)$ центроидов, в чьи компоненты она когда-то входила, а также будем хранить информацию, которая нам будем нужна для ответа на запросы. Например, если мы хотим посчитать минимум на пути, то мы для каждой пары вершина-центроид храним минимум на соответствующем пути. Тогда, при ответе на запрос, мы за $O(\log n)$ операций находим центроид на нужном нам пути и очевидным образом считаем ответ. Так мы можем отвечать на запросы в online и с субьективно более опрятным кодом.

Подробнее смотрите в [конспекте от Сергея Копелиовича](http://acm.math.spbu.ru/~sk1/mm/lections/zksh2016-centroid/conspect.pdf).